# **Using the numerical library _NumPy_**

***numpy_arrays.ipynb***

An example of a problem formulation that uses multi-dimensional *NumPy* arrays and the [xpress.Dot()](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/xpress.Dot.html) operator to formulate constraints simply and efficiently.

&copy; Copyright 2025-2026 Fair Isaac Corporation

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0.
 
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

This example uses FICO&reg; Xpress software. By running it, you agree to the Community License terms of the [Xpress Shrinkwrap License Agreement](https://www.fico.com/en/shrinkwrap-license-agreement-fico-xpress-optimization-suite-on-premises) with respect to the FICO&reg; Xpress software. See the [licensing options](https://www.fico.com/en/fico-xpress-trial-and-licensing-options) overview for additional details and information about obtaining a paid license.

In [ ]:
# Install the xpress package
%pip install -q xpress

Start by importing the *NumPy* and *xpress* packages, create two random matrices and an instance of an Xpress problem.

In [ ]:
import xpress as xp
import numpy as np

rndseed = 10
np.random.seed(rndseed)

A = np.random.random(30).reshape(6, 5)  # A is a 6x5 matrix
Q = np.random.random(25).reshape(5, 5)  # Q is a 5x5 matrix

p = xp.problem()

Add a *NumPy* array of variables. By using [problem.addVariables()](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/problem.addVariables.html) with an integer argument, a *NumPy* array of variables is created.

A term with the identity matrix multiplied by four is then added to the matrix stored in object $Q$.

In [2]:
x = p.addVariables(5)     # Create Numpy array of 5 variables

Q += 4 * np.eye(5)        # Add 4 * the identity matrix to the previous Q matrix

In the code cell below:
  - The <tt>Lin_sys</tt> object contains six linear constraints originating from the *dot* product between the matrix $A$ and vector $x$, with the right-hand side of each constraint passed as an *NumPy* array.

  - <tt>Conv_c</tt> defines a convex quadratic constraint obtained by the *dot* product between the $x$ vector multiplied by itself and by the matrix $Q$.

In [3]:
Lin_sys = xp.Dot(A, x) <= np.array([3, 4, 1, 4, 8, 7])  # Creates 6 linear constraints

Conv_c = xp.Dot(x, Q, x) <= 1                           # Creates 1 quadratic constraint

p.addConstraint(Lin_sys, Conv_c)

Add a convex quadratic objective function by using the *dot* product on variables $x$ minus a randomly generated vector, and optimize the problem.

In [ ]:
x0 = np.random.random(5)     # Random vector

p.setObjective(xp.Dot(x-x0, x-x0))

p.optimize()

p.writeProb("numpy.lp")